In [ ]:
#TASK 1 - Install matplotlib and deap
#       - Import all necessary libraries



In [ ]:
# # TODO: Task 2 - Define the number of locations and vehicles

# num_locations = # Define the number of locations (eg 10)

# locations = # Create a list of tuples representing location coordinates -
#             # try to use a random number generator
#             # (x, y) could be between 0 and 100

# depot = # Define the coordinates for the depot

# num_vehicles = # Define the number of vehicles - (eg 3)

In [ ]:
!pip install matplotlib deap

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

In [ ]:
# TODO: Task 2 - Define the number of locations and vehicles

num_locations = 10

locations = [(random.randrange(0, 100), random.randrange(0, 100)) for _ in range(num_locations)]# Create a list of tuples representing location coordinates -
            # try to use a random number generator
            # (x, y) could be between 0 and 100
print(locations)
depot = (50,50)# Define the coordinates for the depot

num_vehicles = 3 # Define the number of vehicles - (eg 3)

[(81, 14), (3, 94), (35, 31), (28, 17), (94, 13), (86, 94), (69, 11), (75, 54), (4, 3), (11, 27)]


In [ ]:
# TASK 3 - Genetic Algorithm Setup
       # - Figure out how to represent an individual OR encode an individual

creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1))
creator.create("Individual", list, fitness=creator.FitnessMin)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
#TASK 4 - Starting with the toolbox and tools

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.indices) # Now creator.Individual should be defined
toolbox.register("population", tools.initRepeat,list, toolbox.individual)


In [ ]:
# TASK 5 and 6


def evalVRP(individual):
  total_distance = 0
  distance = []
  for vehicles in range(num_vehicles):
    route_distance = 0
    current_location = depot
    route_index_end = len(individual) if vehicles == num_vehicles - 1 else individual.index(vehicles)
    for i in range(route_index_start, route_index_end):
      next_location = locations[individual[i]]
      route_distance += np.sqrt((current_location[0] - next_location[0])**2 + (current_location[1] - next_location[1])**2)
      current_location = next_location
      route_distance += np.sqrt((current_location[0] - depot[0])**2 + (current_location[1] - depot[1])**2)
      distance.append(route_distance)
      total_distance += route_distance
      route_index_start = route_index_end
  return total_distance, np.std(distance)

   # TODO: Task 5 - Write the fitness evaluation function
    #              - Return (total distance, standard deviation)
toolbox.register("evaluate", evalVRP)
#TASK 6 - Register the evaluate function in the toolbox

In [ ]:
def evalVRP(individual):
    total_distance = 0
    distances = []  # Track distance traveled by each vehicle for balance calculation

    # Split the list of locations among vehicles, ensuring each starts and ends at the depot
    for i in range(num_vehicles):
        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        # print(vehicle_route)
        # Calculate total distance traveled by this vehicle
        vehicle_distance = 0
        for k in range(len(vehicle_route) - 1):
            vehicle_distance += np.sqrt((vehicle_route[k][0] - vehicle_route[k + 1][0])**2 + (vehicle_route[k][1] - vehicle_route[k + 1][1])**2)

        total_distance += vehicle_distance
        distances.append(vehicle_distance)

    balance_penalty = np.std(distances)  # Use standard deviation of distances as a penalty for imbalance among vehicles
    return total_distance, balance_penalty
a = evalVRP([num_vehicles])
print(a)
toolbox.register("evaluate", evalVRP)

(79.32212806020776, 37.39280976634709)


In [ ]:
#SOLUTION

toolbox.register("mate", tools.cxPartialyMatched)  # Register the crossover function suitable for permutation-based representation
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)  # Register the mutation function to shuffle indices with a 5% chance per index
toolbox.register("select", tools.selTournament, tournsize=3)  # Register the selection function using tournament selection

In [ ]:
# SOLUTION - TASK 8
def plot_routes(individual, title="Routes"):
    plt.figure()
    # Plot locations as blue dots and the depot as a red square
    for (x, y) in locations:
        plt.plot(x, y, 'bo')
    plt.plot(depot[0], depot[1], 'rs')

    # Draw routes for each vehicle
    for i in range(num_vehicles):
        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        plt.plot(*zip(*vehicle_route), '-', 'rs')

    plt.title(title)
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()

In [ ]:
# Running the Genetic Algorithm
# Running the Genetic Algorithm
def main():
    random.seed(42)  # Seed for reproducibility
    pop = toolbox.population(n=300)  # Generate initial population
    hof = tools.HallOfFame(1)  # Hall of Fame to store the best individual

    # Setup statistics to track
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    # Run the genetic algorithm
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 300, stats=stats, halloffame=hof)

    # Plot the best route found
    plot_routes(hof[0], "Optimal Route")
    return pop, stats, hof

if __name__ == "__main__":
    main()

TypeError: initRepeat() missing 1 required positional argument: 'n'